# Aligning Drum Tabs with Music

This Jupyter Notebook is the product of Thomas Hymel. The contents pertain to the subject of one aspect of an Automatic Drum Transcription (ADT) project. Its main purpose is to perform data manipulation to prepare text-based drum tabs to be aligned with their music track to produce a data point that could be added to a growing number of points in a dev set. 

**I am making the assumption that the onset times that can be mathematically derived from the drum tabs, and then aligned with the labels in the tabs, are good enough for the purpose of training an automatic drum tab algorithm.** This assumption may not turn out to be true, or at least strong enough for an algorithm to work well enough. 

There are many potential hurdles to overcome in this task.
1. Format inconsistency
2. Quality inconsistency
3. Classification specificity inconsistency 
4. Availability of Source Songs
5. Alignment issues with respect to tempo changes, triplets, other tabbing artifacts

There is no single master format that every user follows that allows for tab-to-tab consistency, but there should be enough shared similarities in the starting templates that users often begin with to manually, and ideally automatically with scripts, change the tabs to a single master format. For example, given the tab example above, I could imagine **a type of master format where each 16th note column is a single row of data in a Pandas dataframe, and where the dataframe columns represent each possible drum piece.** The values in the dataframe themselves denote the labels of each data entry. After a consistent labelling scheme is applied to all the rows, the entire tab is aligned to an audio file of the song. At that point the audio file is diced into equal-time-length sections, according to the song length and BPM in the tab, and each tiny 50-100 ms section (X input) is associated with its dataframe tab row (Y output labels). 

I will quickly discuss and summarize the five major challenges I have identified in this task. 

### Challenge 1: Format Inconsistency
This is probably the most challenging hurdle to gathering a significant amount of tab-aligned-to-music data, and requires the most amount of manual labor and attention to fix. Tabs have consistent notations for a *few* things: the character used to denote a blank ('-'), the character used to denote measure splits ('|'), and some type of time-keeping line, normally below each set of measures. Most tabs have the cymbal hits as 'x' or 'X', and the drum hits as 'o' or 'O'. Other than that... there's really no standard. Although tab processing can do some amount of the work required in correcting format inconsistency, it certainly can't do everything. For example, some tabs utilize the musical equivalent of a "repeated section" that is replayed. Normally this type of thing will be denoted by *words* that tell the reader to repeat a certain section. Any tab being processed will need to be manually changed into the proper pre-processing form so that the code can automatically deal with the rest of it. The closer that a tab is to the correct format the faster it will be to manually change it before going into the automatic processing. 

### Challenge 2: Quality inconsistency
Even if a tab is in the perfect format from the beginning, it could be not helpful because the quality of the tab is poor, or at least not up to the standard that I hold for my tabs. Whenever I tab, I listen to the album version of a song and make sure I get every single drum hit, sometimes using videos of live performances to help with the trouble spots. However, not everyone who freely tabs and uploads to the internet will have these standards. Due to the lack of data overall, if I do end up gathering a large amount of other tabs I will most likely make the assumption that any other tab is up to the same standard as mine and accept it as equal. Beggars can't be choosers after all. 

### Challenge 3: Classification specificity inconsistency
Even if a tab is in perfect format, and each hit is recorded properly, there's a chance that the specificity of each drum hit isn't classified to the degree that I normally tab. For example, a cymbal hit on the top of a crash cymbal (like one would play the ride cymbal) I usually denote as 'x', while a crash cymbal edge hit (the way crash cymbals are normally hit) I denote as 'X'. I believe this type of specificity is ultimately out of the scope of this type of project, as only just a few weeks ago the first data set was released that had velocity information in the data set. Regardless of the loudness of a drum hit, or even the difference between a flam and normal hit on a snare drum, a machine learning algorithm should be able to recognize it. The solution to this challenge is that I will collapse the different drum tab classifications into one class for each drum and most cymbals (ride cymbal and hi-hat are more complex). 

### Challenge 4: Availability of Source Songs
All the songs I have tabbed and uploaded are definitely owned by someone who has some type of copyright or proper ownership for it. As such, I would never be able to freely distribute the songs themselves, but only the tabs and the information related to how to line up the album versions of songs (as most songs are exactly the same across different digital platforms) could be shared. Personally, I will be able to procure all the album versions of the tabs used, so this challenge is only a problem when it relates to distribution. This project probably won't be viewed by anyone anyway so this isn't a real challenge.

### Challenge 5: Alignment issues with respect to tempo changes, triplets, and other tabbing artifacts
This challenge is a problem that I don't know how to solve, even with manual attention. In the music that I listen to, and popular music in general, tempo changes are rare. Tempo change refers to the beats per minute of the song changing at some point in the song. This project *operates* under the assumption that a 16th note grid represents the *same timing* for each 16th note, so that if you figure out the timing of one (the resolution) then it can be applied to every 16th note after that. However tempo changes change all of that. Tempo changes will be extremely non-trivial to code to be automatically handled properly. They aren't denoted on tabs except for things like "tempo slows here" or other equivalent statements. To handle a tempo change properly you have to introduce a new column into a tab to represent the BPM of that point in the song, and figure out the math involved to automatically give each point the correct time length. However even this solution isn't robust, since certain songs have a type of continuous tempo change when transitioning between two tempos. 

Triplets are a more common feature in drums and music across all genres, and, luckily, more likely to be handled automatically in code than tempo changes (albeit still with some difficulties). Triplets occur when music or drum onsets occur at different intervals than the expected 16th note grid. For example, instead of having two notes evenly spaced out across 4 16th notes, you could have *three* notes *evenly* spaced out across 4 16th notes. This situation is impossible to capture on an evenly spaced grid, so normally the time-keeping line of that specific measure is changed from something like '4e+a' to something like '4tl ', so that the triplet still takes up the same amount of space in the grid but only contains 3 "valid" positions where notes can be placed. Code would have to be changed slightly to account for this, as the onset times, and resolution of the underlying grid, of these three hits could be properly calculated using only the BPM and the amount of "replacement" that is occuring. Another solution to the triplet challenge would be to throw out triplets whenever they are encountered. The probably account for less than 1% of all drum hits in the music I listen to. An even easier, but less effective, solution would be to simply leave them in and not change anything to account for triplets. Yes, you would have a few problem data points from these triplets, but the effort to get them in (or even to eliminate them) might not be worth it. 

As I am filling this out, I'm not sure what other tabbing artifacts can pop up. I will update this section whenever I encounter some tabbing artifact that makes automatic processing more difficult. 


## Tab Processing: Human-Friendly text to Machine-Friendly text
**The first step is to take in a drum tab's human-friendly text and turn it into machine-friendly text.** A human-friendly drum tab text file is read into Python and printed out below. This tab was recently fully completed by me, and I will be using it as the test case for my code from now on because it is an "ideal" song for the beginning of this exercise. The following properties make it "ideal": 
+ BPM is constant throughout the song ==> no tempo changes
+ no triplets anywhere in the song ==> no tabbing grid resolution weirdness
+ tabbed by me ==> high quality ensured and classification consistency with my previous tabs
+ I own the song ==> access to high quality music track ensured.

**I plan on the machine-friendly text to look very similar to human-friendly text but all measures are on ONE single music line (as opposed to a bunch of music lines spanning multiple pages).** I imagine this format would be much more efficient to import into a Pandas dataframe because it will essentially be a column-named table already. Note that in real life when talking about music I normally use the terminology "line" to refer to 4 measures, but as the word "line" is used often in code to refer to lines of code, I will attempt to use music line when I am referring to a grouping of 4 measures in a human-friendly text tab (but I will probably slip up often). 

In [2]:
# open our tab text file we will work with and see what it looks like, then close it.
with open('The Dark (WVNDER) tab txt.txt', 'r') as example_tab_file:
    print(example_tab_file.read())

Artist: WVNDER
Song: The Dark
Drummer: Brett Schleicher
Tabbed by: Epoch0
BPM: 160

Cymbals:
C = regular crash
C2 = thin crash, higher sounding than C
R = Ride
HH = hihat	
|-x-|: Hit (normal)
|-X-|: Accented strike; If on hihat, loose/washy hihat
|-b-|: Bell hit
|-o-|: On hi-hat, hit while open 
|-s-|: On hi-hat, stomp close on this beat (to make the click sound with foot)

Drums:
B = bass
S = snare
sT, mT, FT = small tom, medium tom, floor tom
|-o-|: normal hit
|-O-|: accented strike
|-f-|: flam
|-g-|: ghost

Guitar Intro: (0:00)
B |----------------|----------------|----------------|----------------|
  |1e+a2e+a3e+a4e+a|1e+a2e+a3e+a4e+a|1e+a2e+a3e+a4e+a|1e+a2e+a3e+a4e+a|

B |----------------|----------------|----------------|----------------|
  |1e+a2e+a3e+a4e+a|1e+a2e+a3e+a4e+a|1e+a2e+a3e+a4e+a|1e+a2e+a3e+a4e+a|

Full Music Intro (0:12)
mT|--o-------------|----------------|o---------------|----------------|
FT|----o-----------|----------------|o---f---o-------|----------------|
R |--

Things to note about the **human-friendly format** that may be useful to consider later while coding:
+  There's usually a "dictionary" to describe the exact notation used for the tab before the tab actually begins. It may be possible to automatically grab this information and fix problematic tabs by converting them to a master format, but more than likely this will be a manual process for any inidividual tab. That is, a unique dictionary probably needs to be made to "convert" the drum piece labelling from their own system to the master format dictionary before any Python script can be run on it. 
+ Text description of the sections of music with their timestamps may be provided before any music line of measures. (e.g. Guitar Intro (0:00)) as seen in the tab above. These will more than likely be discarded in the machine-friendly text. Because of their ability to introduce errant characters that may mess with further processing, they will need to be deleted, or, if the information proves useful later, somehow removed and ignored. 
+ One or two character drum piece labels, usually in capital letters, are used to denote the drums in the tab itself. If the space character is included, all may be described as two character drum piece labels. **Importantly, no case-sensitive single character drum piece label overlaps with any drum hit classifier character.** This fact is essential for creating automatic scripts relying on searching for characters and executing conditionals based on those characters. 
+ The beginning chars of any drum piece music line denotes the drum piece label associated with that line. This fact can be used to find and build up the rows of drum pieces.
+ If a drum piece *is not* used in a single music line (4 measures) then it *is not written* in that music line. However, when appending rows from different music lines together, this missing section must be accounted for by a series of blanks (represented by dashes ---) equal to the length of that particular line (because not all music lines are 4 measures).
+ Measures are split up by the vertical line character `|` and are ultimately not needed in the machine-friendly format, as they don't represent any passing of time. They generally are useful in making the text look nice to humans, so they will probably still in until the very last step where they may be simply removed.
+ The 16th note counting (1e+a2e+a3e+a4e+a) is present at the bottom of *every* music line. This row can be used as a starting or stopping reference.
+ The BPM may or may not be provided by the text file, but will ultimately be needed later on for the mathematical derivation of the resolution of the musical grid.
+ Residual extra white space in the form of multiple empty lines may be included from the fact that human-friendly entire musical lines needs to appear on the same page: if it so happened that a line was unable to fit completely on a page because of the end of the page, it would need to be bumped to the next page with a bunch of returns. 

### Building up the Functions for Tab Pre-Processing
First we construct a high level function that will set the structure for how a text file goes from human-friendly to machine-friendly. After that, the individual helper functions are coded and tested. 

In [3]:
def hf_to_mf(tab_file_name, tab_char_labels):
    """
    High level function that takes in a tab's text file string name referring to a text file of human-friendly drum tabulature
    and a Python dictionary describing that tab's drum piece labels 
    Returns a machine-friendly tab representation in a Python array
    """
    # establish list of master format drum piece char labels using a function
    master_format_dict = get_master_format_chars()
    
    # establish dictionary of drum piece labels to convert to master format
    tab_conversion_dict = produce_label_mapping(master_format_dict, tab_char_labels)
    
    # replace the 2 char labels used in the tab given by tab_file_name with the master format 2 char labels 
    tab = convert_labels(tab_file_name, tab_conversion_dict)
    
    # change the tab to include "white space" drum piece lines whenever there are tab lines WITHOUT a drum piece label
    expanded_tab = expand_tab(tab, master_format_dict)
    
    # align all the drum piece lines in the entire tab with each other into one long line for every drum piece line
    mf_tab = combine_lines(expanded_tab, master_format_dict)
    
    return mf_tab

#### 1) get_master_format_chars function
Let's now build a function to get the master format drum piece character labels. I have decided to use a master format that corresponds similarly to ADT literature-established short-character codes, but limiting myself to only 2 characters for the reason that normally in drum tabs there are 2 characters used at the beginning of a row to denote which drum piece that row is for. 

In [4]:
def get_master_format_chars():
    """
    Used to define the master format drum piece character labels. This function will be manually hard coded.
    Returns a dictionary, whose keys are a text string that describes that drum piece label 
    and whose values are the two character string of labels according to the master format (defined by this function)
    """
    
    master_format_dict = {'bass drum'     :'BD',
                          'snare drum'    :'SD',
                          'high tom'      :'HT',
                          'mid tom'       :'MT',
                          'low tom'       :'LT',
                          'hi-hat'        :'HH',
                          'ride cymbal'   :'RD',
                          'crash cymbal'  :'CC',
                          'crash cymbal 2':'C2',     # extra cymbals if needed, classification will ultimately collapse later
                          'crash cymbal 3':'C3',     # extra cymbals if needed, classification will ultimately collapse later
                          'crash cymbal 4':'C4',     # extra cymbals if needed, classification will ultimately collapse later
                          'splash cymbal' :'SC',
                          'china cymbal'  :'CH'
                         }
    
    return master_format_dict

#### 2) produce_label_mapping function

Now we build a function that defines the transfering from one drum piece character label to the master format labels. This function will take in the label dictionary given to us by the hf_to_mf high level function and output the mappings of the labels themselves. After, we'll test it with two dictionaries to make sure it behaves correctly. Note that this function assumes that all keys in the master format dictionary will show up in the tab's character labels dictionary, even if that tab doesn't actually use all those drum pieces. 

In [5]:
def produce_label_mapping(master_format_dict, tab_char_labels):
    """
    Takes in the master format label dictionary, whose keys are drum descriptions and whose values are the 2 char labels,
    and the tab's label dictionary, whose keys are the exact same drum descriptions and whose values are the 2 char labels
    that occur in the current tab being passed to hf_to_mf
    Returns a dictionary, whose keys are the TAB's 2 char labels and whose values are the MASTER FORMAT 2 char labels
    """
    
    tab_conversion_dict = {}     # empty dict object to be constructed
    
    for key in tab_char_labels:
        tab_conversion_dict[tab_char_labels[key]] = master_format_dict[key]  # makes the assumption that all keys in one dictionary is the same as the the other

    return tab_conversion_dict

In [6]:
# Testing the produce_label_mapping function's capabilities
input_dict1 = {'my': 'first', 'dogs': 'name', 'was': 'Melanie'}
input_dict2 = {'my': 'second', 'dogs': 'breed', 'was': 'boxer'}
print(produce_label_mapping(input_dict1,input_dict2))

{'second': 'first', 'breed': 'name', 'boxer': 'Melanie'}


#### 3) convert_labels function

We now have the mapping that takes the labels used in the given drum tab to the master format labels, so the next step is to go through the tab and replace the labels in the tab with the master formal labels. This step is the first one that deals with manipulating the entire tab text file so there may be useful helper functions to be created within it. 

In [7]:
def convert_labels(tab_file_name, tab_conversion_dict):
    """
    Takes in the tab's file name (string) and the tab conversion dictionary and uses them to find and replace the tab's labels,
    whenever they occur at the beginning of a line, with the master format labels (values of the dictionary)
    Returns a python list of lines with the master format labels now???
    """
    with open(tab_file_name, 'r') as tab_file:    # opens the tab_file_name file and closes it after this block of code is done
        tab_to_convert = tab_file.readlines()     # returns a python array of the text file with each line as a string, including the \n new line character at the end of each string, in the order in which it appears in the file
    
    tab = []   # build this array tab up and return it
    
    for line in tab_to_convert:                 # loop over each element in tab (code line in tab)
        temp = len(tab)                         # toggle to check later if the following block of code ever added an element to tab
        for key in tab_conversion_dict:          # At each line, loop over each key in the tab_conversion_dict
            if line.startswith(key):             # If a line starts with the key, reassign it to be the new label from dict...
                tab.append(tab_conversion_dict[key] + line[len(key):])   # ...and concatenate with the rest of that line
        if temp == len(tab):                  # if the len(tab) never changed, we didn't find a line.startswith(key), but we still want to keep that line
            tab.append(line)

    return tab     #returns the python list of lines (still with \n at the end) but with the correct master format labels, in the correct order

In [8]:
# Testing the main part of the convert_labels function
test_lines =  ['this', 'is', 'an', 'important', 'test', 'for', 'this', 'thirsty', 'funny', 'function']
test_dict = {'th' : 'osmos', 'fu': 'pu'}
test_output = []
for line in test_lines:
    temp = len(test_output)
    for key in test_dict:
        if line.startswith(key):
            test_output.append(test_dict[key] + line[len(key):])
    if temp == len(test_output):
        test_output.append(line)

print(test_output)

['osmosis', 'is', 'an', 'important', 'test', 'for', 'osmosis', 'osmosirsty', 'punny', 'punction']


#### 4) expand_tab function and subfunctions get_special_char and reverse_then_split_on_tk
The tab is now in an array of strings, with each element representing a single line in from the original tab text file, except that the labels are consistent with the master format. Great! In a human-friendly tab, if a drum piece is not used in a music line then it is not included in that music lines drum piece labels. But in a machine-friendly tab, we want the computer to know explicitly that there isn't anything there with blanks (here, the character -). 

The idea of the expand function is the following: use the master format labels to scan the beginning of lines, noting if any label appears. Then, for each music line that occurs (normally denoted by the time-keeping line of '  |1ea+a...'), we add above that time-keeping line all the drum piece label lines that do NOT occur already immediately above that time-keeping line. In this way every time-keeping line will contain immediately above it a drum piece label line that corresponds to the length of the time-keeping line. I expect some subfunctions to be made for this expand_tab function.

A special character getter function was made for certain formatting characters so that if any other function needs to use one of these characters it could grab it that way instead of hard-coding them in multiple different functions. 

In a future function (not in expand_tab) we will rearrange all the music lines together to get one single music line.

In [9]:
def expand_tab(tab, master_format_dict):
    """
    Takes in a tab (ordered array of strings format, with master format drum piece labels applied) and the master format dictionary
    and builds in the empty drum piece labels that will be necessary to append the different music lines together in a later function
    Returns the expanded tab in some format???
    """
    # Use the get_special_chars function to get the needed special chars
    tk_label, measure_char, blank_char = get_special_chars()
    # tk_label is the time-keeping label is two spaces, used to denote the beginning of a time-keeping line
    # measure_char is the character used to split the measures and otherwise make the tabulature human readable
    # blank_char is the character used to denote a blank in a drum piece line 

    tk_chars = tk_label + measure_char   # the time-keeping line starts with two chars (here, two spaces) and then the measure_char
    
    # grab a list of all drum pieces used at least once in the tab
    drum_pieces_used = []                             # empty array to be appended
    for line in tab:                                  # loop over every line in tab
        for value in master_format_dict.values():     # loop over all the values in master_format_dict, which are the 2 char labels
            if line.startswith(value + measure_char) and value not in drum_pieces_used: 
                drum_pieces_used.append(value)      # line must start with the 2 char label+measure_char AND NOT already be in the list        

    # change the structure of the tab object so one can more easily expand tab later
    split_on_tk_reversed = reverse_then_split_on_tk(tab)
    
    # now we have an array of strings, where each string either begins with tk_chars, or is the first element in the array
    # For each element that starts with tk_chars, we check which drum piece labels exist in that element, and
    # add in the appropriate length empty line for each missing drum piece, and then combine
    # This is handled by the subfunction add_empty_lines
    expanded_split_on_tk_reversed = add_empty_lines(split_on_tk_reversed, drum_pieces_used, tk_label, measure_char, blank_char)
    
    # To make the future appending function easier, we want to sort the drum piece labels so that each tk line has the same ordered drum lines after.
    ordered_ex_split_on_tk_reversed = order_tk_lines(expanded_split_on_tk_reversed, drum_pieces_used, tk_label, measure_char)
    
    # now we want to undo the structural changes made to the tab
    combined_ordered_ex_reversed = ''.join(ordered_ex_split_on_tk_reversed)          # combine the tk split elements into one string
    ordered_expanded_reversed_tab = combined_ordered_ex_reversed.splitlines(keepends=True) # separate into different individual lines, keeping the newline character
    
    expanded_tab = list(reversed(ordered_expanded_reversed_tab))    # reverse the tab entirely
    
    return expanded_tab

In [10]:
def get_special_chars():
    """
    Hard-coded getter function to get the following special characters when needed. Normally these characters are the same 
    for any tab, but would need to be changed if a tab doesn't follow these conventions
    returns three outputs in this order: time-keeping 2 character label; 
    the measure character (1 character string); and the blank character (1 character string)
    """
    tk_label = '  '
    measure_char = '|'
    blank_char = '-'
    
    return tk_label, measure_char, blank_char

In [11]:
def reverse_then_split_on_tk(tab):
    """
    Takes in a tab that is in normal order, split along newlines, with the newline characer still at the end
    Returns a tab that is in reverse order, split along time-keeping lines, with the time-keeping chars put back
    into the appropriate position
    """
    tk_label, measure_char, blank_char = get_special_chars()
    
    tk_chars = tk_chars = tk_label + measure_char   # the time-keeping line starts with two chars (here, two spaces) and then the measure_char
    
    reversed_tab = list(reversed(tab))                       # make a reverse tab object so we can execute the next code more easily with time-keeping lines coming before the drum piece lines
    combined_reversed_tab = ''.join(reversed_tab)              #combine it into one string so we can split on the time-keeping chars
    split_on_tk_reversed = combined_reversed_tab.split(tk_chars) # split into parts based on the tk chars
    for idx in range(1, len(split_on_tk_reversed)):             # for loop to add the delimiter tk chars back into the proper string elements
        split_on_tk_reversed[idx] =  tk_chars + split_on_tk_reversed[idx] # append the tk chars at the beginning of the proper elements
        
    return split_on_tk_reversed

In [12]:
# testing the portion of the code that grabs the drum pieces used
tab =  ['this', 'is', 'an', 'important', 'test', 'for', 'this', 'thirsty', 'funny', 'function']
master_format_dict = {'th' : 'th', 'fu': 'fu'}
measure_char = ''
drum_pieces_used = []                             # empty set to be appended
for line in tab:                                  # loop over every line in tab
    for value in master_format_dict.values():     # loop over all the values in master_format_dict, which are the 2 char labels
        if line.startswith(value + measure_char) and value not in drum_pieces_used: 
            drum_pieces_used.append(value)      # line must start with the 2 char label+measure_char AND NOT already be in the list
                
print(list(reversed(drum_pieces_used)))

['fu', 'th']


In [13]:
# testing the portion of the code that splits a list and adds back in the delimiter in the correct place
tab =  ['this\n', 'is\n', 'an\n', 'important\n', 'test\n', 'for\n', 'this\n', 'thirsty\n', 'funny\n', 'function\n']
joined_tab = ''.join(tab)
print(joined_tab)

split_tab = joined_tab.split('\n')
print(split_tab)
for i in range(0,len(split_tab)-1):
    split_tab[i] = split_tab[i] + '\n'

print(split_tab)

this
is
an
important
test
for
this
thirsty
funny
function

['this', 'is', 'an', 'important', 'test', 'for', 'this', 'thirsty', 'funny', 'function', '']
['this\n', 'is\n', 'an\n', 'important\n', 'test\n', 'for\n', 'this\n', 'thirsty\n', 'funny\n', 'function\n', '']


##### 4a) add_empty_lines subfunction
This function takes in the reversed, combined and then split on the time-keeping characters (with time-keeping characters added back in where they were deleted) tab, the drum pieces used in the tab, the time-keeping label, the measure character, and the blank character. For each element in the array, we check if it starts with the time-keeping characters. If it does, we do a bunch of things, like find the length of the time keeping line, find the drum piece labels that are already in that section, add in the missing lines, and then combine it all back into one string that starts on the time-keeping characters. 

In [14]:
def add_empty_lines(split_on_tk_reversed, drum_pieces_used, tk_label, measure_char, blank_char):
    """
    Adds the necessary empty lines to ensure all time-keeping music lines have the full amount of drum piece label lines that
    should exist in the tab
    Returns an array of strings, most of which start on the tk chars, with each one of those containing after it either a new
    empty drum piece label line (the same length as the tk line), or a drum piece label line that already existed there
    """
    
    tk_chars = tk_label + measure_char     # useful variable
    
    expanded_split_on_tk_reversed = []     # build up this array to be returned
    
    for string in split_on_tk_reversed:    # loop through all elements of the array
        if not string.startswith(tk_chars):    # if element does not start with tk_chars, we don't want to change it
            expanded_split_on_tk_reversed.append(string)  # immediately append it, and then move onto next string
        else:                                  # in the case where element starts with tk_chars
            drums_in_tk = []                   # build up the drums already found in this tk element
            for drum in drum_pieces_used:      # loop over all drums found in the drum tabs
                if (drum + measure_char) in string:  # checks if the string of drums+measure_char is in the tk element
                    drums_in_tk.append(drum)         # if so, add it to the list of drums in the tk
            drums_not_in_tk = list(x for x in drum_pieces_used if x not in drums_in_tk) # grabs all the drum pieces not in tk element but used in overall drum tab
            
            tk_line = string[len(tk_label) : string.find('\n')]   # grabs the entirety of the time-keeping line, without the \n character at the end of it
            blank_line = ''                      # build up a blank line string equal in length to tk_line (without the initial tk label or the \n char)
            for char in tk_line:                 # loop through chars in current tk line
                if char is measure_char:         # if any of the chars are the measure char, keep it there
                    blank_line = blank_line + measure_char
                else:                            # if any of the chars are anything but measure chars, "replace" it with blank char
                    blank_line = blank_line + blank_char
            
            blank_drums = ''                     # the entire set of drum piece lines in one string
            for drum in drums_not_in_tk:         # utilize the correct length blank line to add the correct blank lines for each drum
                blank_drum_line = ''.join((drum, blank_line, '\n'))  # construct the blank drum piece line, adding the new line character
                blank_drums = blank_drums + blank_drum_line          # add this blank drum piece line to running blank drums string
            expanded_string = string[:string.find('\n')+1] + blank_drums + string[string.find('\n')+1:] # use str slicing to make a new expanded string, placing the new blank drum lines immediately after the tk line
            expanded_split_on_tk_reversed.append(expanded_string)          # append the expanded tk string to the running array.
    
    return expanded_split_on_tk_reversed

In [15]:
#testing add_empty_lines function

split_on_tk_test = ['  |1e+a2e+a|1e+a2e+a3e+a|\nBD|o---o---|--o---o---o-|\nHH|x-x-x-x-|x-x-x-x-x-x-|\n', '  |1e+a2e+a|1e+a2e+a|\nBD|o---o---|--o---o-|\n', "Artist: WVNDER\nTabber:Me\n"]
dpu_test = ['BD', 'SD', 'HH', 'LT', 'CC']
tk_label_test = '  '
measure_test = '|'
blank_test = '-'

print("This is what the input looks like before add_empty_lines function\n")
for item in split_on_tk_test:
    print(item)

test_ael = add_empty_lines(split_on_tk_test, dpu_test, tk_label_test, measure_test, blank_test)
print("This is what the output of the add_empty_lines function is:\n")
for item in test_ael:
    print(item)

This is what the input looks like before add_empty_lines function

  |1e+a2e+a|1e+a2e+a3e+a|
BD|o---o---|--o---o---o-|
HH|x-x-x-x-|x-x-x-x-x-x-|

  |1e+a2e+a|1e+a2e+a|
BD|o---o---|--o---o-|

Artist: WVNDER
Tabber:Me

This is what the output of the add_empty_lines function is:

  |1e+a2e+a|1e+a2e+a3e+a|
SD|--------|------------|
LT|--------|------------|
CC|--------|------------|
BD|o---o---|--o---o---o-|
HH|x-x-x-x-|x-x-x-x-x-x-|

  |1e+a2e+a|1e+a2e+a|
SD|--------|--------|
HH|--------|--------|
LT|--------|--------|
CC|--------|--------|
BD|o---o---|--o---o-|

Artist: WVNDER
Tabber:Me



##### 4b,c) order_tk_lines subfunction and get_desired_order subfunction
At this point we have the tab in the expanded version, but in an array split on the time-keeping lines, while also being reversed (so the time-keeping lines come before the drum piece lines, which also means the entire tab in reversed as well.) Now we want to order the drum piece lines with respect to each other in different time-keeping lines. 

This order doesn't have to be alphabetical. For making the machine-friendly text even more human-friendly, I made a subfunction called get_desired_order that is simply a hard-coded order of the master format labels that the coder would like to see the drum piece labels ordered in (counting from bottom to top)

In [16]:
def order_tk_lines(expanded_split_on_tk_reversed, drum_pieces_used, tk_label, measure_char):
    """
    Returns the array in the same structure but all the drum piece lines are in the same order with respect to each other in 
    different time-keeping lines
    """
    tk_chars = tk_label + measure_char     # useful variable
    
    # If you don't want to use alphabetical order, but some other order for your drums, the function get_desired_order
    # will return a dictionary that contains the order key based on master format labels and the coder's (my) liking
    desired_order_dict = get_desired_order()
    drum_pieces_used.sort(key=desired_order_dict.get)    # sorting drum pieces used based on the desired order dictionary
    
    ordered_tk_lines = []     # build up this array to be returned
    
    for string in expanded_split_on_tk_reversed:    # loop through all elements of the array
        if not string.startswith(tk_chars):         # if element does not start with tk_chars, we don't want to change it
            ordered_tk_lines.append(string)         # immediately append it, and then move onto next string
        else:                                       # in the case where element starts with tk_chars
            exploded_tk_string = string.splitlines(keepends=True) # splits the tk elements on the newline character, and keeps it   
            dct = {x+measure_char: i for i, x in enumerate(drum_pieces_used)}  # setting up a key dictionary to be used later to compare and sort ("BD|" = 0, "CC|" = 1, etc.)
            drums_in_etk_string = [x for x in exploded_tk_string if x[:len(tk_chars)] in dct]  # grabs only the lines in exploded tk string that contains drum piece labels
            drums_in_etk_string.sort(key = lambda x: dct.get(x[:len(tk_chars)]))               # sort the drum line strings by the 
            iterator = iter(drums_in_etk_string)              #create the iterator for the next line
            ordered_exploded_tk_string = [next(iterator) if x[:len(tk_chars)] in dct else x for x in exploded_tk_string]  # orders the drum piece lines without affecting any other line
            ordered_tk_string = ''.join(ordered_exploded_tk_string)                       # connects the exploded strings back together
            ordered_tk_lines.append(ordered_tk_string)                            # appends the new ordered string to the running array
    
    return ordered_tk_lines

In [17]:
def get_desired_order():
    """
    This function grabs the desired order of the drum piece labels relative to themselves. In a normal human-friendly tab,
    the time-keeping line is at the bottom. The numbers here represent counting from the bottom up
    Returns the dictionary that contains the master format labels as keys and the ordered in integer numbers as values
    """
    desired_order_dict = {'BD': 1,
                          'SD': 2,
                          'HT': 9,
                          'MT': 8,
                          'LT': 7,
                          'HH': 3,
                          'RD': 4,
                          'CC': 5,
                          'C2': 6,     # extra cymbals if needed, classification will ultimately collapse later
                          'C3': 12,     # extra cymbals if needed, classification will ultimately collapse later
                          'C4': 13,     # extra cymbals if needed, classification will ultimately collapse later
                          'SC':11,
                          'CH':10
                         }
    
    return desired_order_dict

In [18]:
# test for order_tk_lines subfunction
dpu_test = ['BD', 'SD', 'HH', 'LT', 'CC']
tk_label_test = '  '
measure_test = '|'
blank_test = '-'

for item in test_ael:
    print(item)

test_otkl = order_tk_lines(test_ael, dpu_test, tk_label_test, measure_test)

for item in test_otkl:
    print(item)

  |1e+a2e+a|1e+a2e+a3e+a|
SD|--------|------------|
LT|--------|------------|
CC|--------|------------|
BD|o---o---|--o---o---o-|
HH|x-x-x-x-|x-x-x-x-x-x-|

  |1e+a2e+a|1e+a2e+a|
SD|--------|--------|
HH|--------|--------|
LT|--------|--------|
CC|--------|--------|
BD|o---o---|--o---o-|

Artist: WVNDER
Tabber:Me

  |1e+a2e+a|1e+a2e+a3e+a|
BD|o---o---|--o---o---o-|
SD|--------|------------|
HH|x-x-x-x-|x-x-x-x-x-x-|
CC|--------|------------|
LT|--------|------------|

  |1e+a2e+a|1e+a2e+a|
BD|o---o---|--o---o-|
SD|--------|--------|
HH|--------|--------|
CC|--------|--------|
LT|--------|--------|

Artist: WVNDER
Tabber:Me



##### 5) combine_lines function
This function takes in an expanded tab, in a human-friendly ordered array of lines, but with all the drum piece lines sorted according to the coder and including blank lines for whenever a music line didn't include drum piece hits/labels in the original tab. It also takes in the master format dictionary so that it can know which drum piece labels to look for. 

There's going to be two extra lines in this machine-friendly text, and both lines will be located above the music line. One line, the top line, will be a concatenation of all the *non machine friendly lines* that normally come before a tab. Ultimately this is done simply to preserve information that is in the tab. The second extra line will be the lines that exist right above a music line, if it has any text, ideally placed in the same position relative to the measures. 

In [19]:
def combine_lines(expanded_tab, master_format_dict):
    """
    Takes in an expanded tab in human-friendly readable ordered array, and the master format dictionary, and 
    then makes the drum line into one large line
    Returns the tab in machine-friendly text, where each drum piece line is one long line, still with the measure characters in
    """
  
    # Prelim- Get the special characters because we'll need one of them
    tk_label, measure_char, blank_char = get_special_chars()
    tk_chars = tk_label + measure_char
    
    # grab the drum pieces used in the piece, and then sort them by the desired order.
    # This could probably be more efficiently done considering every music line now contains all of drum pieces used, but whatever
    drum_pieces_used = []                             # empty array to be appended
    for line in expanded_tab:                                  # loop over every line in tab
        for value in master_format_dict.values():     # loop over all the values in master_format_dict, which are the 2 char labels
            if line.startswith(value + measure_char) and value not in drum_pieces_used: 
                drum_pieces_used.append(value)      # line must start with the 2 char label+measure_char AND NOT already be in the list        
    desired_order_dict = get_desired_order()
    drum_pieces_used.sort(key=desired_order_dict.get)    # sorting drum pieces used based on the desired order dictionary
   
    # First we'll get rid of any duplicate, consecutive newline's that are ultimately not needed, to simplify processing later
    exp_tab = [x for i,x in enumerate(expanded_tab) if i ==0 or x != '\n' or x != expanded_tab[i-1]]
    
    # Now we'll make the assumption that there's only one line of useful text above any music line, if any, and later attempt to 
    # append it to the correct spot above that measure in the eventual single long music line.
    
    # reverse and split on the time-keeping chars so we have an array of strings, almost all of which start with tk_chars
    exp_sotk_rev_tab = reverse_then_split_on_tk(exp_tab)
    
    num_drums = len(drum_pieces_used)      # total number of DRUM piece lines that are on attached underneath each tk line

    mf_tab = []         # final array to build up. Should have 1 tk line, num_drums num of lines, and then 1 notes on drums line, and then 1 garbage text line
    extra_lines = 3     # HARD CODED number of lines 
    total_lines = num_drums + extra_lines 
    
    for idx in range(total_lines):   # as described above: 1tk line, then drum lines, then note line, then garbage line.
            mf_tab.append('')
    
    for string in exp_sotk_rev_tab:         # loop over all the tk elements
        if string.startswith(tk_chars):           # if the string starts with tk_chars, then it contains all the drum piece lines "underneath" it, already in order
            exploded_tk_string = string.splitlines(keepends=False) # splits the tk elements on the newline character, loses the newline because we don't need it anymore 
            total_chars_in_tk_line = len(exploded_tk_string[0])    # grab this for later use
            for line_num, line in enumerate(exploded_tk_string):   # grabs the index and line of every line in a tk element
                if line_num < total_lines-2:        # accessing a tk line or drum piece line 
                    mf_tab[line_num] = line[len(tk_chars):] + mf_tab[line_num]   # get a slice of after the first 3 chars, and append it to the beginning, because we are going backwards
                elif line_num == total_lines-2:     # accessing a note line
                    mf_tab[line_num] = line + (" " * (total_chars_in_tk_line - len(line) - len(tk_chars))) + mf_tab[line_num]    # fills in the note line with whitespace so that the other note line notes will align properly
                else:                               # in a case where the tk element had more drum_piece + extra lines than expected, so the rest can be added to garbage line
                    mf_tab[-1] = line + " " + mf_tab[-1]
        else:                                    # the case where our tk element did not start with tk_chars
            mf_tab[-1] = string + " " + mf_tab[-1]    # it gets added to the garbage line
    
    # Now we add back in the important line labels at the beginning of each long line
    for idx in range(len(mf_tab)):  # loop through each long, combined line in mf_tab
        if idx == 0:                                
            mf_tab[idx] = tk_chars + mf_tab[idx]    # start the tk line with the tk_chars
        elif idx < 1+num_drums:                 # will enter this a total number of drums times
            mf_tab[idx] = drum_pieces_used[idx-1] + measure_char + mf_tab[idx]  # start the drum line with the drum pieces used 2 chars + measure_char
        elif idx == (num_drums+1):   # we are on note line
            mf_tab[idx] = " "*len(tk_chars) + mf_tab[idx]  # adds spaces*tk_chars to realign all the notes to their proper places 
    
    mf_tab_final = list(reversed(mf_tab))   # back into the "human friendly" version of the mostly machine friendly text: Garbage line, note line, and then desired order until tk line
    
    return mf_tab_final

In [67]:
# testing the entire hf_to_mf function

tab_file_name = 'The Dark (WVNDER) tab txt.txt'
tab_char_labels = { 'bass drum'     :'B ',
                    'snare drum'    :'S ',
                    'high tom'      :'sT',
                    'mid tom'       :'mT',
                    'low tom'       :'FT',
                    'hi-hat'        :'HH',
                    'ride cymbal'   :'R ',
                    'crash cymbal'  :'C ',
                    'crash cymbal 2':'C2',     # extra cymbals if needed, classification will ultimately collapse later
                    'crash cymbal 3':'C3',     # extra cymbals if needed, classification will ultimately collapse later
                    'crash cymbal 4':'C4',     # extra cymbals if needed, classification will ultimately collapse later
                    'splash cymbal' :'SC',
                    'china cymbal'  :'CH'
                    }

mf_output = hf_to_mf(tab_file_name, tab_char_labels)

mf_txt = []  #creating new object because we will use mf_output later on)
for idx in range(len(mf_output)):
    mf_txt.append(mf_output[idx] + "\n")

with open('hf_to_mf_output.txt', 'w') as test_out:
    test_out.write(''.join(mf_txt))
    
for idx in range(len(mf_output)):
    print(len(mf_output[idx]))

print(mf_output)

524
2672
2672
2672
2672
2672
2672
2672
2672
2672
2672
['Artist: WVNDER Song: The Dark Drummer: Brett Schleicher Tabbed by: Epoch0 BPM: 160  Cymbals: CC= regular crash C2 = thin crash, higher sounding than C RD= Ride HH = hihat\t |-x-|: Hit (normal) |-X-|: Accented strike; If on hihat, loose/washy hihat |-b-|: Bell hit |-o-|: On hi-hat, hit while open  |-s-|: On hi-hat, stomp close on this beat (to make the click sound with foot)  Drums: BD= bass SD= snare HT, mT, FT = small tom, medium tom, floor tom |-o-|: normal hit |-O-|: accented strike |-f-|: flam |-g-|: ghost          ', '   Guitar Intro: (0:00)                                                                                                                    Full Music Intro (0:12)                                                                                                                 Verse 1 (0:24)                                                                                                                              

##### **It works!** 

Although it isn't obvious from the output given above, as the string wraps at the end of the width of the window, the text file saved and opened in a proper text editor shows that indeed each drum piece line is one continuous line, stacked in the correct order, with blanks where they should be and the drum tab data where it should be. Hooray!

The output of the function hf_to_mf is the following, in this order: 1 "garbage" line that contains a concatenation of all the non-tab or music line information inside the text file (for example, the header information or the normal drum piece label legend); 1 "note line" that is the line directly above any music line that may contain useful notes about the tab, while also being properly aligned with the measures throughout; n number of lines where n is the total number of drum pieces used in the entire tab; and 1 time-keeping line as the last line. This object is now a matrix, or labeled table, or whatever you want to call it. The point is that it now is ready to be loaded into a pandas dataframe object to more easily manipulate it in the future. 

## Importing into a Pandas Dataframe and Alignment with Music

We are now ready to import pandas and work with dataframe objects. The process tab is currently using the master format dictionary, so that will come in handy for giving names to the columns in the tab (the tab will be transposed from its current form). 

In [68]:
# import numpy and pandas modules
import numpy as np
import pandas as pd

## Quick initial code to show what I might expect a dataframe to look like:
tab_df_quick = pd.DataFrame()

for idx, line in enumerate(mf_output):
    if(idx !=0):
        npline = np.array(list(line))
        tab_df_quick.insert(idx-1, "line"+ str(idx), npline)

print(tab_df_quick)

     line1 line2 line3 line4 line5 line6 line7 line8 line9 line10
0              M     L     C     C     R     H     S     B       
1              T     T     2     C     D     H     D     D       
2              |     |     |     |     |     |     |     |      |
3        G     -     -     -     -     -     -     -     -      1
4        u     -     -     -     -     -     -     -     -      e
...    ...   ...   ...   ...   ...   ...   ...   ...   ...    ...
2667           -     -     -     -     -     -     -     -      4
2668           -     -     -     -     -     -     -     -      e
2669           -     -     -     -     -     -     -     -      +
2670           -     -     -     -     -     -     -     -      a
2671           |     |     |     |     |     |     |     |      |

[2672 rows x 10 columns]


### Building up the Functions for Tab Alignment with Music

Just as I did with the machine-friendly tab converter, I will make a high level function that controls the flow of the functions for these sections.  

In [69]:
def align_tab_with_music(mf_output, song_data):
    """
    Takes in the output of the hf_to_mf function (here, called mf_output) and a song_data object that represents
    the information needed to align the tab with the song properly (TBD LATER)
    Returns a pandas dataframe that contains the input data from a song sliced into the 16th note grid in one column,
    and then all the other columns that represent the classifications of the onsets of the drum pieces for that input data row
    """
    
    # Use the machine-friendly text output to get the array of strings into a dataframe, naming the table columns with the key or values of master format dict
    tab_df = tab_to_dataframe(mf_output)
    
    # Use the song object data to import the song into a Python object so it can be manipulated
    song = songify(song_data)
    
    # Use the tab dataframe and the song, along with other information, to align the tab with the music!
    df_MAT = combine_tab_and_song(tab_df, song)
    
    # df_MAT = dataframe object of the music aligned tab (with the music attached as well)
    df_MAT = None  
    
    
    return df_MAT

##### 1) tab_to_dataframe function
The tab_to_dataframe function is the first function in the high level aligning tab function. Its job is to take the output from the hf_to_mf function (an array of strings, with each string being one long line) and convert it into a pandas dataframe. 

In [70]:
def tab_to_dataframe(mf_output):
    """
    Takes in the machine-friendly tab (an array of strings) so that it can be put in a dataframe
    Returns the dataframe, with the columns named from the master format dictionary
    """
    
    master_format_dict = get_master_format_chars()  # grabbing the master format dictionary
    tk_label, measure_char, blank_char = get_special_chars() # grab the special chars
    tk_chars = tk_label + measure_char
    
    
    mf_reversed = list(reversed(mf_output))  # the reverse of this so that the tk line is on top, or in column 0 of the dataframe 
    
    tab_df = pd.DataFrame()
    
    for idx, line in enumerate(mf_reversed):    # going through the machine-friendly reversed Python array for the last time
        line_chars = line[:len(tk_label)]       # grab the first chars used to identify the line
        npline = np.array(list(line[len(tk_label):]))
        if line_chars == tk_chars:               # time-keeping line
            add_df = pd.DataFrame({'Time-Keeping': npline})
            tab_df.insert(idx, 'Time-Keeping', npline)
        else:
            None

        #tab_df.concat
        #tab_df_quick.insert(idx-1, "line"+ str(idx), npline)
    
    return tab_df

In [71]:
# testing tab_to_dataframe function using 

tab_df_test = tab_to_dataframe(mf_output)
print(tab_df_test)

Empty DataFrame
Columns: []
Index: []


### TO DO LIST
+ ~~Set up individual Jupyter Notebook for Aligning Drum Tabs with Music~~ (Apr 27)
+ ~~Successfully convert one tab from human-friendly to machine-friendly format using only Python scripts~~ (May 1st)
  + ~~Make methods in Python to manipulate the tab text~~ (Apr 28th, 29th, 30th, May 1st)
+ ~~Finish the text description of the Challenge sections~~ (May 1st)
+ ~~Put this on GitHub / learn how to update and integrate this to GitHub~~ (May 2nd)
+ Import an example song mp3 into Python object 
+ Successfully import a tab into a pd dataframe 
+ Successfully align a tab with a pd dataframe 
+ Align multiple tabs (at least 10) in a standardized semi-automatic way


#### List of Useful Shortcuts

* Ctrl + shift + P = List of Shortcuts
* Enter (command mode) = Enter Edit Mode (enter cell to edit it)
* Esc (edit mode) = Enter Command Mode (exit cell)
* A = Create Cell above
* B = Create Cell below
* D,D = Delete Cell
* Shift + Enter = Run Cell (code or markdown)
* M = Change Cell to Markdown
* Y = Change Cell to Code
* Ctrl + Shift + Minus = Split Cell at Cursor
